![title](img/this-is-fine-spark.jpeg)

## 🔥 Spark fires 🔥 - file formats are not created equal

... or CSV and JSON be like meh 👎

In this scenario, we will look at why file-formats matter, and specifically why you want to avoid formats like CSV and JSON where possible. 

We'll even count rows. Not because it's exciting but because you end up doing it more than you'd imagine due to data reconciliation, etc. 😴

Note, for this scenario we will have two options:
1. Use the bike-data datasets, which are a little on the small side.
2. Generate some **synthetic data which will take up ~ 30G on your hard drive**.

*Consider yourself warned!* 😅

### Bootstrapping

In [ ]:
import os
import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.5.0  pyspark-shell'

spark = (
    SparkSession
    .builder.master("spark://spark:7077")
    # .config("spark.eventLog.enabled", "true")
    # .config("spark.eventLog.dir", "/data/tmp/spark-events")
    .appName("spark-fires-more-cores-than-partitions")
    .getOrCreate()
)

# option 1 - use small data - uncomment to use
# input_data_path_original = '/data/bike-data/201508_trip_data.csv'
# input_data_path_csv = '/data/bike-data/201508_trip_data_underscored.csv'
# input_data_path_json = '/data/bike-data/201508_trip_data.json'

# # note, we supply a schema to avoid schema inference OOM issues with JSON
# schema = schema = StructType([
#     StructField("TripID", StringType(), False),
#     StructField("Duration", IntegerType(), False),
#     StructField("StartDate", TimestampType(), False),
#     StructField("StartStation", StringType(), False),
#     StructField("StartTerminal", StringType(), False),
#     StructField("EndDate", TimestampType(), False),
#     StructField("EndStation", StringType(), False),
#     StructField("EndTerminal", StringType(), False),
#     StructField("BikeNum", IntegerType(), False),
#     StructField("SubscriberType", StringType(), False),
#     StructField("ZipCode", StringType(), False)
# ])

# option 2 - use bigger data which requires a little more legwork to generate
# (note, you'll need ~ 30G or so free for this)
input_data_path_csv = '/data/G1_1e8_1e8_100_0.csv'
input_data_path_json = '/data/G1_1e8_1e8_100_0.json'

# note, we supply a schema to avoid schema inference OOM issues with JSON
schema = schema = StructType([
    StructField("id1", StringType(), False),
    StructField("id2", StringType(), False),
    StructField("id3", StringType(), False),
    StructField("id4", IntegerType(), False),
    StructField("id5", IntegerType(), False),
    StructField("id6", IntegerType(), False),
    StructField("v1", IntegerType(), False),
    StructField("v2", IntegerType(), False),
    StructField("v3", DecimalType(), False)
])

input_data_path_parquet = '/data/bike-data-parquet'
input_data_path_avro = '/data/bike-data-avro'

spark.version

### Let's prep our data

So for this we have two options:
1. Use a pre-existing but pretty small dataset.
1. Do a little more legwork to create a larger synthetic dataset, which will better demonstrate the differences between the formats.

I will shoot for option 2, but if you are feeling lazy feel free to take the easier option. 

#### Option 1 - the small dataset 👎

We are going to borrow some test data from the excellent _Spark, The Definitive Guide_ Git repo. 

Make sure to go back to the cell that creates the session and switch up the input file paths for this to work.

In [ ]:
# !mkdir -p /data/bike-data
# !wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_station_data.csv -P /data/bike-data
# !wget https://raw.githubusercontent.com/udacity/data-analyst/master/projects/bike_sharing/201508_trip_data.csv -P /data/bike-data

In [ ]:
!ls -lh /data/bike-data

Then we are going to write the CSV data down in the following additional formats: JSON, Parquet and Avro. 

*Note, for the CSV and JSON formats we will have just a single input file - as this is how you will often encounter these formats in the wild, e.g. if no-one could be bothered to use a better format they probably also could not be bothered to write down multiple file-splits.*

In [ ]:
df = pd.read_csv(input_data_path_original)
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace('#', 'Num')  # note, we remove the spaces from the column names as Avro will not deal with those : o
df.to_csv(input_data_path_csv, index=False)

#### Option 2 - the bigger dataset

For this we are going to use [falsa](https://github.com/mrpowers-io/falsa), a synthetic data generation tool created by Matthew Powers. 

To use this you will need to follow this rough approach:
1. Create a new venv, with your preferred tool, e.g. pyenv, etc.
2. Install falsa as per the README.
3. Then run the following command from your spark-fires directory:

`
falsa groupby --path-prefix=./data --size MEDIUM
`

After running this you should see that you have a fairly chunky file with about 6.4G and 100k rows of data. Result. 

#### Let's just write down the JSON, Parquet and Avro versions of the CSV
*Note, this take about 4 mins on my machine for the bigger synthetic dataset*

In [ ]:
%%time

tmp_data_path_json = '/data/tmp-json'

df = spark.read.option("header", True).schema(schema).csv(input_data_path_csv)
df.repartition(1).write.format('json').mode('overwrite').save(tmp_data_path_json)

Okay, let's rename our file-split that Spark has created, to make things easier to manage. *Look away friends, just a bit of filename hacking going on* 🙈

In [ ]:
from pathlib import Path

dir_itr = Path(tmp_data_path_json).iterdir()

file_path = next(dir_itr)
while os.path.isdir(file_path) or str(file_path).endswith('.crc'):
    file_path = next(dir_itr)

file_path
os.rename(file_path, input_data_path_json)

Let's sort the Parquet and Avro - note this will take > 5 mins.

In [ ]:
# !rm -rf /data/bike-data-parquet
# !rm -rf /data/bike-data-avro

In [ ]:
%%time

def get_csv():
    return spark.read.option("header", True).schema(schema).csv(input_data_path_csv)

if not os.path.exists(input_data_path_parquet):
    df = get_csv()
    df.repartition(12).write.format('parquet').save(input_data_path_parquet)

if not os.path.exists(input_data_path_avro):
    df = get_csv()
    df.repartition(12).write.format('avro').save(input_data_path_avro)

### Data formats at rest 💾

#### ... digging into the details

Okay, before we do any processing let's have a look at what these formats look like on disk, at rest. There can't be that much difference right? 🤷

In [ ]:
!ls -lh /data/bike-data

Note, the false synthetic data should be a CSV file named **G1_1e8_1e8_100_0.csv**, if you have gone with the default option 2 dataset.

In [ ]:
!ls -lh /data

In [ ]:
!ls -lh /data/bike-data-avro

In [ ]:
!du -sh /data/bike-data-avro

In [ ]:
!ls -lh /data/bike-data-parquet

In [ ]:
!du -sh /data/bike-data-parquet

#### ... say whaaat 🔥🔥🔥 🚒 🚒 🚒 🧯🧯🧯

Ahem, so the first thing we can see is that there is a reasonable difference in the space taken up by these formats:
 * CSV ~ 6.5G
 * JSON ~ 10G
 * Parquet ~ 2.1G
 * Avro ~ 3.1G

So with respect to the original CSV dataset:
 * JSON is > 50% bigger
 * Parquet is ~ 70% smaller
 * Avro is > 50% smaller

In this instance, Parquet is a clear winner. It is obviously worth noting this will vary massively with each dataset. For instance, with the original bike-data dataset the **CSV is a whopping x4 on-disk and JSON is x8** compared with the Parquet - oof. Not ideal for your object storage bills. 😮 💰💰💰🔥🔥🔥

The binary formats, Parquet and Avro, clearly have smaller footprints. Parquet will generally have an advantage on a bigger dataset which contains more enumerations due to columnar column encoding (which is a bit beyond the scope of this notebook to go into, you can [read more here though in Alex Merced's excellent overview](https://medium.com/data-engineering-with-dremio/all-about-parquet-part-06-encoding-in-parquet-optimizing-for-storage-b857ebfcf9a9).

### Now let's do some data processing

#### First off let's just count the rows
Don't forget to **first restart the kernel**, so we don't have any cached file references, metadata, etc.

In [ ]:
%%time 

df = spark.read.csv(input_data_path_csv)
df.count()

In [ ]:
%%time 

# note, we supply a schema to avoid schema inference OOM issues with JSON
df = spark.read.schema(schema).json(input_data_path_json)
df.count()

In [ ]:
%%time 

df = spark.read.parquet(input_data_path_parquet)
df.count()

In [ ]:
%%time 

df = spark.read.format('avro').load(input_data_path_avro)
df.count()

#### Uh, counting be like *wa wa waaa* 🔥🔥🔥 🚒 🚒 🚒 🧯🧯🧯

So even counting shows a considerable difference between the file formats:
 * CSV ~ 20 secs
 * JSON ~ 36 secs (although we gave it a hand by supplying a schema)
 * Parquet ~ 1.6 secs
 * Avro ~ 15 secs

The clear a winner here as **Parquet is ~ 9 - 22 times faster than the other formats**. This is because Parquet stores various pieces of metadata in file footers so that the counts can be determined by reading a very small amount of the files.

#### Okay, let's throw in a filter with our count
Don't forget to **first restart the kernel**, so we don't have any cached file references, metadata, etc.

In [ ]:
# df.summary('min', '25%', '50%', '75%', 'max').show()

In [ ]:
bounds_filter = F.col('v3').between(47, 52)

In [ ]:
%%time 

df = spark.read.option('header', 'true').schema(schema).csv(input_data_path_csv)
df.filter(bounds_filter).count()

In [ ]:
%%time 

df = spark.read.schema(schema).json(input_data_path_json)
df.filter(bounds_filter).count()

In [ ]:
%%time 

df = spark.read.parquet(input_data_path_parquet)
df.filter(bounds_filter).count()

In [ ]:
%%time 

df = spark.read.format('avro').load(input_data_path_avro)
df.filter(bounds_filter).count()

#### Parquet wins the filter wars 🔥🔥🔥 🚒 🚒 🚒 🧯🧯🧯

So with a filter thrown in the counts break down as follows:
 * CSV ~ 36 secs
 * JSON ~ 29 secs (although we gave it a hand by supplying a schema)
 * Parquet ~ 2.5 secs
 * Avro ~ 17 secs

The clear a winner here again as **Parquet is ~ 7 - 15 times faster than the other formats**. This is footer metadata previously mentioned and columnar storage layout which minimizes the data read. Boom, Parquet for the win. 

#### Right let's do some actual processing
Don't forget to **first restart the kernel**, so we don't have any cached file references, metadata, etc.

In [ ]:
def agg_the_data(df):
    df.groupby('id1').avg('v3').show()

In [ ]:
%%time 

df = spark.read.option('header', 'true').schema(schema).csv(input_data_path_csv)
agg_the_data(df)

In [ ]:
%%time 

df = spark.read.schema(schema).json(input_data_path_json)
agg_the_data(df)

In [ ]:
%%time 

df = spark.read.parquet(input_data_path_parquet)
agg_the_data(df)

In [ ]:
%%time 

df = spark.read.format('avro').load(input_data_path_avro)
agg_the_data(df)

### ... and the winner is ... 🏆🏆🏆

So basic agg breaks down as follows:
* CSV ~ 43 secs
* JSON ~ 38 secs 
* Parquet ~ 4 secs
* Avro ~ 21 secs

So again Parquet is the clear winner. **Parquet is ~ 5 to 10 times faster** on this task which is because of filter pushdown and it's columnar layout. This results in the Parquet tasks reading very much less data.

Obviously, these differences will vary considerably with datasets. Our test datasets have few columns, if we were had a hundred of more columns in play the differences in performance would be monumental. Also, remember this is all local, so we are missing network and object storage latencies, which would again favour Parquet.

### In summary
So what can we say in summary? Formats like CSV and JSON are not *bad* per se. But binary, and particularly columnar formats, have far superior performance characteristics in the analytics space where we are often targeting computation at a subset of partitions, rows and columns. 

As a row-based format, Avro is commonly seen in stream-processing where we are reading and processing whole records at a time and where columnar formats offer no advantage.

Parquet dominated all of our tests by quite some way. This is why it is the defacto standard format of modern data processing, and is utilised under the hood by the newer data lake formats like Apache Iceberg and Delta Lake.

In [ ]:
# spark.stop()